In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
import numpy as np
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Pad images from 28x28 to 32x32 (LeNet-5 expects 32x32)
x_train = np.pad(x_train, ((0, 0), (2, 2), (2, 2)), 'constant')
x_test = np.pad(x_test, ((0, 0), (2, 2), (2, 2)), 'constant')

# Normalize pixel values
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.

# Reshape to add channel dimension
x_train = x_train.reshape(-1, 32, 32, 1)
x_test = x_test.reshape(-1, 32, 32, 1)

# One-hot encode labels
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

print(f"Training shape: {x_train.shape}, Test shape: {x_test.shape}")

In [ ]:
def build_lenet5(input_shape=(32, 32, 1), num_classes=10):
    model = models.Sequential(name='LeNet-5_Custom')

    # C1: Convolutional layer (6 filters, 5x5 kernel)
    model.add(layers.Conv2D(filters=6, kernel_size=(5, 5), activation='tanh', input_shape=input_shape))
    model.add(layers.AveragePooling2D(pool_size=(2, 2)))

    # C2: Convolutional layer (16 filters, 5x5 kernel)
    model.add(layers.Conv2D(filters=16, kernel_size=(5, 5), activation='tanh'))
    model.add(layers.AveragePooling2D(pool_size=(2, 2)))

    # C3: Convolutional layer (120 filters, 5x5 kernel)
    model.add(layers.Conv2D(filters=120, kernel_size=(5, 5), activation='tanh'))

    # Flatten before Fully Connected Layers
    model.add(layers.Flatten())

    # F4: Fully connected layer (84 units)
    model.add(layers.Dense(units=84, activation='tanh'))

    # F5: Output layer (10 classes)
    model.add(layers.Dense(units=num_classes, activation='softmax'))

    return model

# Instantiate model
model = build_lenet5()
model.summary()

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
history = model.fit(
    x_train, y_train,
    batch_size=128,
    epochs=10,
    validation_split=0.2,
    verbose=2
)

In [ ]:
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f"Test accuracy: {test_acc:.4f}")


In [ ]:
plt.figure(figsize=(14, 5))

# Accuracy
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Acc')
plt.plot(history.history['val_accuracy'], label='Val Acc')
plt.title('Accuracy over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

# Loss
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.title('Loss over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.show()

In [ ]:
def plot_predictions(model, x_data, y_true, n=10):
    preds = model.predict(x_data[:n])
    pred_classes = np.argmax(preds, axis=1)
    true_classes = np.argmax(y_true[:n], axis=1)

    plt.figure(figsize=(15, 3))
    for i in range(n):
        plt.subplot(1, n, i+1)
        plt.imshow(x_data[i].reshape(32, 32), cmap='gray')
        plt.title(f"T: {true_classes[i]}, P: {pred_classes[i]}")
        plt.axis('off')
    plt.show()

plot_predictions(model, x_test, y_test)
